# IE 598 Course Demo 2: Optimization for LASSO 

LASSO is widely used in Machine Learning and Statistics for sparse regression.

Given data $(x_1,y_1),\ldots, (x_N,y_N)$, where $x_i\in R^d, y_i\in R$, $i=1,\ldots,N$, the LASSO model aims to solve the optimization problem
$$\min_{w} F(w):=\frac{1}{2}\sum_{i=1}^N (x_i^Tw-y_i)^2+\lambda\|w\|_1$$
where $\lambda\geq 0$ is the regularization parameter. 

In this demo, we illustrate and compare some of the algorithms learned in this module (subgradient descent, Nesterov's smoothing, proximal gradient, and accelerated gradient methods to solve LASSO and investigate their empirical peformances.

In [ ]:
%matplotlib inline 
import numpy as np
from matplotlib import pyplot as plt

## Data Generation

In [ ]:
N = 100
dim = 30
lamda = 1/np.sqrt(N);
np.random.seed(50)
w = np.matrix(np.random.multivariate_normal([0.0]*dim, np.eye(dim))).T
X = np.matrix(np.random.multivariate_normal([0.0]*dim, np.eye(dim), size = N))
y = X*w

L = (np.linalg.svd(X)[1][0])**2
print(L)
max_iter = 100

## Optimization via CVXPY

CVX is a Matlab-based modeling system for convex optimization problems. http://cvxr.com/cvx/

CVXPY is a Python-embedded modeling language for convex optimization problems. 
http://www.cvxpy.org/en/latest/index.html 

The default solver is EOCS (Embedded Conic Solver). See detials here.  https://www.embotech.com/ECOS


In [ ]:
import cvxpy as cvx

w = cvx.Variable(dim)
loss = cvx.sum_squares(X*w-y)/2 + lamda * cvx.norm(w,1)

problem = cvx.Problem(cvx.Minimize(loss))
problem.solve(verbose=True) 
opt = problem.value
print('Optimal Objective function value is: {}'.format(opt))

## Optimization Algoirthms for Nonsmooth Convex Minimization


In [ ]:
## Define the objective, subgradient oracle, proximal operator, and gradient for smoothed function (Huber)  
def obj(w):
    r = X*w-y;
    return np.sum(np.multiply(r,r))/2 +  lamda * np.sum(np.abs(w))

def subgrad(w):
    return  X.T*(X*w-y) + lamda*np.sign(w) 

def f_grad(w):
    return  X.T*(X*w-y) 

def soft_threshod(w,mu):
    return np.multiply(np.sign(w), np.maximum(np.abs(w)- mu,0))  

def smooth_grad(w, mu):
    temp = np.multiply((np.abs(w)<=mu),w/mu) + np.multiply((np.abs(w)>mu),np.sign(w)) 
    return X.T*(X*w-y) + lamda * temp


### Subgradient Descent

In [ ]:
## Subgradient Descent
w = np.matrix([0.0]*dim).T
obj_SD = []
gamma = 0.01
for t in range(0, max_iter):
    obj_val = obj(w)
    w = w - gamma * subgrad(w)/np.sqrt(t+1)
    
    obj_SD.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective error vs. iteration (log scale)
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
plt.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.legend(prop={'size':12})
plt.xlabel('Iteration')
plt.ylabel('Objective error')

### Gradient Descent for Nesterov's Smoothing 

In [ ]:
# Nesterov's Smoothing
w = np.matrix([0.0]*dim).T
smoothness = 0.01
obj_SM = []
for t in range(0, max_iter):
    obj_val = obj(w)
    w = w - smooth_grad(w, smoothness)/(L + lamda/smoothness)
    
    obj_SM.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective error vs. iteration (log scale)
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
plt.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
plt.legend(prop={'size':12})
plt.xlabel('Iteration')
plt.ylabel('Objective')

### Proximal Gradient

In [ ]:
## Proximal Gadient 
w = np.matrix([0.0]*dim).T
obj_PG = []
for t in range(0, max_iter):
    obj_val = obj(w)
    w = w - (1/L)* f_grad(w)
    w= soft_threshod(w,lamda/L)
    
    obj_PG.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective error vs. iteration (log scale)
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
plt.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
plt.semilogy(t, np.array(obj_PG)-opt, 'b', linewidth = 2, label = 'Proximal Gradient')
plt.legend(prop={'size':12})
plt.xlabel('Iteration')
plt.ylabel('Objective error')

### Proximal Gradient with Backtracking Line-Search 

In [ ]:
## Proximal Gadient with Line Search
w = np.matrix([0.0]*dim).T
obj_PG_LS = []
LL=1
gamma = 1/LL
beta = 1.2
for t in range(0, max_iter):
    obj_val = obj(w)
    w_prev = w
    delta = 1
    while (delta>1e-3):
        gamma = 1/LL
        w = w_prev - gamma * f_grad(w_prev)    
        w = soft_threshod(w,lamda * gamma)
        delta = obj(w) - obj_val - f_grad(w_prev).T*(w-w_prev)- (LL/2) * np.linalg.norm(w-w_prev)**2
        LL = LL*beta
    LL = LL/beta   
    
    obj_PG_LS.append(obj_val.item())

In [ ]:
## Plot objective error vs. iteration (log scale)
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
plt.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
plt.semilogy(t, np.array(obj_PG)-opt, 'b--', linewidth = 2, label = 'Proximal Gradient')
plt.semilogy(t, np.array(obj_PG_LS)-opt, 'b', linewidth = 2, label = 'Proximal Gradient (backtracking)')
plt.legend(prop={'size':12})
plt.xlabel('Iteration')
plt.ylabel('Objective error')


### Accelerated Proximal Gradient

In [ ]:
## Nesterovs' Accelerated Proximal Gradient
w = np.matrix([0.0]*dim).T
v = w
obj_APG = []
gamma = 1/L
for t in range(0, max_iter):
    obj_val = obj(w)
    w_prev = w
    w = v - gamma * f_grad(v)
    w = soft_threshod(w,lamda * gamma)
    v = w + t/(t+3) * (w - w_prev)

    obj_APG.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective vs. iteration
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
ax.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
ax.semilogy(t, np.array(obj_PG)-opt, 'b', linewidth = 2, label = 'Proximal Gradient')
ax.semilogy(t, np.array(obj_APG)-opt, 'c', linewidth = 2, label = 'Accelerated Proximal Gradient')
ax.legend(prop={'size':12})
ax.set_xlabel('Iteration')
ax.set_ylabel('Objective error')

### Accelerated Proximal Gradient with Backtracking Line-Search 

In [ ]:
## Nesterovs' Accelerated Proximal Gradient with Backtracking
w = np.matrix([0.0]*dim).T
v = w
obj_APG_LS = []
L=1
gamma = 1/L
beta = 1.2
for t in range(0, max_iter):
    obj_val = obj(w)
    w_prev = w
    delta = 1
    while (delta>1e-3):
        gamma = 1/L
        w = v - gamma * f_grad(v)    
        w = soft_threshod(w,lamda * gamma)
        delta = obj(w) - obj_val - f_grad(w_prev).T*(w-w_prev)- (L/2) * np.linalg.norm(w-w_prev)**2
        L = L*beta
    L = L/beta    
    v = w + t/(t+3) * (w - w_prev)

    obj_APG_LS.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective vs. iteration
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
ax.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
# ax.semilogy(t, np.array(obj_PG)-opt, 'b--', linewidth = 2, label = 'Proximal Gradient')
ax.semilogy(t, np.array(obj_PG_LS)-opt, 'b', linewidth = 2, label = 'Proximal Gradient (backtracking)')
# ax.semilogy(t, np.array(obj_APG)-opt, 'c--', linewidth = 2, label = 'Accelerated Proximal Gradient')
ax.semilogy(t, np.array(obj_APG_LS)-opt, 'c', linewidth = 2, label = 'Accelerated Proximal Gradient (backtracking)')
ax.legend(prop={'size':12})
ax.set_xlabel('Iteration')
ax.set_ylabel('Objective error')

### Douglas Rachford Splitting/ADMM 


In [ ]:
## ADMM
w = np.matrix([0.0]*dim).T
z = w
u = w
obj_ADMM = []
rho = 5
for t in range(0, max_iter):
    obj_val = obj(w)
    w = np.linalg.inv((X.T)*X + rho*np.identity(dim))*(X.T*y + rho*z - u )
    z= soft_threshod(w + u/rho,lamda/rho)
    u = u + rho * (w-z)
    obj_ADMM.append(obj_val.item())
    if (t%5==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))

In [ ]:
## Plot objective vs. iteration
t = np.arange(0, max_iter)
fig, ax = plt.subplots(figsize = (9, 6))
ax.semilogy(t, np.array(obj_SD)-opt, 'g', linewidth = 2, label = 'Subgradient Descent')
plt.semilogy(t, np.array(obj_SM)-opt, 'm', linewidth = 2, label = 'Nesterov Smoothing')
ax.semilogy(t, np.array(obj_PG)-opt, 'b', linewidth = 2, label = 'Proximal Gradient')
ax.semilogy(t, np.array(obj_APG)-opt, 'c', linewidth = 2, label = 'Accelerated Proximal Gradient')
ax.semilogy(t, np.array(obj_ADMM)-opt, 'r', linewidth = 2, label = 'ADMM')

ax.legend(prop={'size':12})
ax.set_xlabel('Iteration')
ax.set_ylabel('Objective error')